In [1]:
from imports import *

# set columnns display format
pd.set_option('display.max_columns', None)
# default pandas decimal number display format
# pd.options.display.float_format = '{:20,.3f}'.format

In [2]:
# import raw, messy data
df = acquire.get_zillow_data()

In [3]:
# clean data
df = prepare.prep_zillow(df)

In [4]:
# scale and split the data
train, validate, test = prepare.split_scale(df)

In [5]:
# adding location clusters A to dataframe
cols = ['latitude', 'longitude', 'zip_bin_insgfnt high', 'zip_bin_insgfnt low',
       'zip_bin_sgfnt high']
kmeans = KMeans(n_clusters=4)
kmeans.fit(train[cols])

train['location_clusters'] = kmeans.predict(train[cols])
validate['location_clusters'] = kmeans.predict(validate[cols])
test['location_clusters'] = kmeans.predict(test[cols])

In [6]:
# adding age cluster A to dataframe
cols = ['age']
kmeans = KMeans(n_clusters=3)
kmeans.fit(train[cols])

train['age_clusters'] = kmeans.predict(train[cols])
validate['age_clusters'] = kmeans.predict(validate[cols])
test['age_clusters'] = kmeans.predict(test[cols])

In [7]:
# adding area clusters to dataframe - somewhat significant

cols =['total_sqft', 'lot_sqft', 'living_sqft']
kmeans = KMeans(n_clusters=3)
kmeans.fit(train[cols])

train['area_clusters'] = kmeans.predict(train[cols])
validate['area_clusters'] = kmeans.predict(validate[cols])
test['area_clusters'] = kmeans.predict(test[cols])

In [8]:
# adding size clusters A to dataframe - not significant

cols = ['bedrooms', 'bathrooms', 'full_bath']
kmeans = KMeans(n_clusters=3)
kmeans.fit(train[cols])
train['size_clusters'] = kmeans.predict(train[cols])
validate['size_clusters'] = kmeans.predict(validate[cols])
test['size_clusters'] = kmeans.predict(test[cols])

In [9]:
# adding value clusters to dataframe - not significant

cols = ['structure_value', 'assessed_value', 'land_value','taxamount']
kmeans = KMeans(n_clusters=3)
kmeans.fit(train[cols])
train['value_clusters'] = kmeans.predict(train[cols])
validate['value_clusters'] = kmeans.predict(validate[cols])
test['value_clusters'] = kmeans.predict(test[cols])

In [10]:
train.head()

,parcelid,bathrooms,bedrooms,total_sqft,living_sqft,full_bath,latitude,longitude,lot_sqft,roomcnt,structure_value,assessed_value,land_value,taxamount,logerror,age,taxrate,county,transaction_month,county_Los Angeles,county_Orange,county_Ventura,zip_code,zip_bin,zip_bin_insgfnt high,zip_bin_insgfnt low,zip_bin_sgfnt high,zip_bin_sgfnt low,location_clusters,age_clusters,area_clusters,size_clusters,value_clusters
36408,11852584,0.166667,0.333333,0.223146,0.223146,0.166667,0.489849,0.722683,0.142086,0.000000,0.249999,0.369648,0.219774,0.190572,0.008887,0.426471,1.512845,Los Angeles,0.750,1,0,0,90640,insgfnt high,1,0,0,0,1,1,0,2,0
38129,12470054,0.166667,0.333333,0.115889,0.115889,0.166667,0.361449,0.689920,0.131308,0.000000,0.064824,0.161569,0.149200,0.080098,-0.032206,0.448529,1.306395,Los Angeles,0.875,1,0,0,90712,insgfnt high,1,0,0,0,1,1,0,2,2
26349,14033839,0.166667,0.333333,0.155691,0.155691,0.166667,0.279580,0.811640,0.130391,0.583333,0.127344,0.445221,0.387142,0.170193,-0.003708,0.338235,1.141333,Orange,0.625,0,1,0,92708,insgfnt low,0,1,0,0,0,1,0,2,0
31869,17194875,0.250000,0.500000,0.418958,0.418958,0.166667,0.605691,0.264698,0.202229,0.666667,0.715666,0.847915,0.352102,0.300011,0.007720,0.095588,1.095202,Ventura,0.750,0,0,1,91320,insgfnt low,0,1,0,0,0,2,1,2,1
24862,11903595,0.166667,0.333333,0.187020,0.187020,0.166667,0.532605,0.693399,0.118126,0.000000,0.135567,0.445165,0.380948,0.178279,0.033695,0.477941,1.195283,Los Angeles,0.625,1,0,0,91803,insgfnt high,1,0,0,0,1,1,0,2,0


In [11]:
train.age_clusters = train.age_clusters.astype('str')
validate.age_clusters = validate.age_clusters.astype('str')
test.age_clusters  = test.age_clusters.astype('str')
train.location_clusters = train.location_clusters.astype('str')
validate.location_clusters = validate.location_clusters.astype('str')
test.location_clusters  = test.location_clusters.astype('str')
train.area_clusters = train.area_clusters.astype('str')
validate.area_clusters = validate.area_clusters.astype('str')
test.area_clusters  = test.area_clusters.astype('str')
train.size_clusters = train.size_clusters.astype('str')
validate.size_clusters = validate.size_clusters.astype('str')
test.size_clusters  = test.size_clusters.astype('str')
train.value_clusters = train.value_clusters.astype('str')
validate.value_clusters = validate.value_clusters.astype('str')
test.value_clusters  = test.value_clusters.astype('str')

In [12]:
cols = ['location_clusters', 'age_clusters', 'area_clusters', 'size_clusters', 'value_clusters']
train_dummy = pd.get_dummies(train[cols], dummy_na=False, drop_first=False)
train = pd.concat([train, train_dummy], axis=1)

validate_dummy = pd.get_dummies(validate[cols], dummy_na=False, drop_first=False)
validate = pd.concat([validate, validate_dummy], axis=1)

test_dummy = pd.get_dummies(test[cols], dummy_na=False, drop_first=False)
test = pd.concat([test, test_dummy], axis=1)

In [13]:
train.head()

,parcelid,bathrooms,bedrooms,total_sqft,living_sqft,full_bath,latitude,longitude,lot_sqft,roomcnt,structure_value,assessed_value,land_value,taxamount,logerror,age,taxrate,county,transaction_month,county_Los Angeles,county_Orange,county_Ventura,zip_code,zip_bin,zip_bin_insgfnt high,zip_bin_insgfnt low,zip_bin_sgfnt high,zip_bin_sgfnt low,location_clusters,age_clusters,area_clusters,size_clusters,value_clusters,location_clusters_0,location_clusters_1,location_clusters_2,location_clusters_3,age_clusters_0,age_clusters_1,age_clusters_2,area_clusters_0,area_clusters_1,area_clusters_2,size_clusters_0,size_clusters_1,size_clusters_2,value_clusters_0,value_clusters_1,value_clusters_2
36408,11852584,0.166667,0.333333,0.223146,0.223146,0.166667,0.489849,0.722683,0.142086,0.000000,0.249999,0.369648,0.219774,0.190572,0.008887,0.426471,1.512845,Los Angeles,0.750,1,0,0,90640,insgfnt high,1,0,0,0,1,1,0,2,0,0,1,0,0,0,1,0,1,0,0,0,0,1,1,0,0
38129,12470054,0.166667,0.333333,0.115889,0.115889,0.166667,0.361449,0.689920,0.131308,0.000000,0.064824,0.161569,0.149200,0.080098,-0.032206,0.448529,1.306395,Los Angeles,0.875,1,0,0,90712,insgfnt high,1,0,0,0,1,1,0,2,2,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,1
26349,14033839,0.166667,0.333333,0.155691,0.155691,0.166667,0.279580,0.811640,0.130391,0.583333,0.127344,0.445221,0.387142,0.170193,-0.003708,0.338235,1.141333,Orange,0.625,0,1,0,92708,insgfnt low,0,1,0,0,0,1,0,2,0,1,0,0,0,0,1,0,1,0,0,0,0,1,1,0,0
31869,17194875,0.250000,0.500000,0.418958,0.418958,0.166667,0.605691,0.264698,0.202229,0.666667,0.715666,0.847915,0.352102,0.300011,0.007720,0.095588,1.095202,Ventura,0.750,0,0,1,91320,insgfnt low,0,1,0,0,0,2,1,2,1,1,0,0,0,0,0,1,0,1,0,0,0,1,0,1,0
24862,11903595,0.166667,0.333333,0.187020,0.187020,0.166667,0.532605,0.693399,0.118126,0.000000,0.135567,0.445165,0.380948,0.178279,0.033695,0.477941,1.195283,Los Angeles,0.625,1,0,0,91803,insgfnt high,1,0,0,0,1,1,0,2,0,0,1,0,0,0,1,0,1,0,0,0,0,1,1,0,0


In [31]:
train.columns

Index(['parcelid', 'bathrooms', 'bedrooms', 'total_sqft', 'living_sqft',
       'full_bath', 'latitude', 'longitude', 'lot_sqft', 'roomcnt',
       'structure_value', 'assessed_value', 'land_value', 'taxamount',
       'logerror', 'age', 'taxrate', 'county', 'transaction_month',
       'county_Los Angeles', 'county_Orange', 'county_Ventura', 'zip_code',
       'zip_bin', 'zip_bin_insgfnt high', 'zip_bin_insgfnt low',
       'zip_bin_sgfnt high', 'zip_bin_sgfnt low', 'location_clusters',
       'age_clusters', 'area_clusters', 'size_clusters', 'value_clusters',
       'location_clusters_0', 'location_clusters_1', 'location_clusters_2',
       'location_clusters_3', 'age_clusters_0', 'age_clusters_1',
       'age_clusters_2', 'area_clusters_0', 'area_clusters_1',
       'area_clusters_2', 'size_clusters_0', 'size_clusters_1',
       'size_clusters_2', 'value_clusters_0', 'value_clusters_1',
       'value_clusters_2'],
      dtype='object')

cols = ['age_clusters_0', 'age_clusters_1', 'age_clusters_2','location_clusters_0', 'location_clusters_1',  'location_clusters_2', 'location_clusters_3','total_sqft','lot_sqft','size_clusters_0','size_clusters_1', 'size_clusters_2', 'value_clusters_0', 'value_clusters_1', 'value_clusters_2']
Polynomial k=2 interaction only
(0.161358448910367, 0.15736544905806119, 0.17749488159077875)

cols = ['age_clusters_0', 'age_clusters_1', 'age_clusters_2','location_clusters_0', 'location_clusters_1',  'location_clusters_2', 'location_clusters_3','total_sqft','lot_sqft','size_clusters_0','size_clusters_1', 'size_clusters_2', 'value_clusters_0', 'value_clusters_1', 'value_clusters_2', transaction_month]
Polynomial k=2
(0.16106119020400828, 0.15747390252723342, 0.17749220299601765)

cols = ['living_sqft', 'structure_value', 'assessed_value', 'land_value',
       'zip_bin_insgfnt high', 'zip_bin_sgfnt high', 'location_clusters_0',
       'location_clusters_2', 'location_clusters_3', 'value_clusters_0']
Multiple regression: 
(multiple_rfe_k=, 7)    0.161640
(multiple_rfe_k=, 6)    0.161650
(multiple_rfe_k=, 4)    0.161820
(multiple_rfe_k=, 5)    0.161820
(multiple_rfe_k=, 3)    0.161996
(multiple_rfe_k=, 2)    0.162445
baseline                0.162624

(multiple_rfe_k=, 7)    0.157336
(multiple_rfe_k=, 6)    0.157388
(multiple_rfe_k=, 3)    0.157633
(multiple_rfe_k=, 4)    0.157658
(multiple_rfe_k=, 5)    0.157658
(multiple_rfe_k=, 2)    0.157899
basline                 0.158068

(multiple_rfe_k=, 6)    0.177436
(multiple_rfe_k=, 7)    0.177443
(multiple_rfe_k=, 4)    0.177781
(multiple_rfe_k=, 5)    0.177781
(multiple_rfe_k=, 3)    0.177918
(multiple_rfe_k=, 2)    0.178179
baseline                0.178470

Polynomial k=2:
(0.16151388904167482, 0.15735778462696057, 0.1774465577764435)

cols = ['living_sqft', 'structure_value', 'assessed_value', 'land_value',
       'county_Los Angeles', 'county_Orange', 'county_Ventura',
       'zip_bin_insgfnt high', 'zip_bin_sgfnt high', 'location_clusters_0',
       'location_clusters_2', 'location_clusters_3', 'age_clusters_0',
       'area_clusters_1', 'area_clusters_2', 'size_clusters_0',
       'value_clusters_0']
(multiple_rfe_k=, 7)    0.161640
(multiple_rfe_k=, 6)    0.161650
(multiple_rfe_k=, 4)    0.161820
(multiple_rfe_k=, 5)    0.161820
(multiple_rfe_k=, 3)    0.161996
(multiple_rfe_k=, 2)    0.162445
baseline                0.162624

(multiple_rfe_k=, 7)    0.157336
(multiple_rfe_k=, 6)    0.157388
(multiple_rfe_k=, 3)    0.157633
(multiple_rfe_k=, 4)    0.157658
(multiple_rfe_k=, 5)    0.157658
(multiple_rfe_k=, 2)    0.157899
basline                 0.158068

(multiple_rfe_k=, 6)    0.177436
(multiple_rfe_k=, 7)    0.177443
(multiple_rfe_k=, 4)    0.177781
(multiple_rfe_k=, 5)    0.177781
(multiple_rfe_k=, 3)    0.177918
(multiple_rfe_k=, 2)    0.178179
baseline                0.178470

Polynomial k=2
(0.16118663037140582, 0.15752054355182657, 0.17757655954543025)


       
cols = ['living_sqft', 'structure_value', 'assessed_value', 'land_value',
       'county_Los Angeles', 'county_Orange', 'county_Ventura',
       'zip_bin_insgfnt high', 'zip_bin_sgfnt high', 'age_clusters_0',
       'area_clusters_1', 'area_clusters_2', 'size_clusters_0']
Multiple regression:
(multiple_rfe_k=, 6)    0.161759
(multiple_rfe_k=, 7)    0.161759
(multiple_rfe_k=, 5)    0.161809
(multiple_rfe_k=, 4)    0.161819
(multiple_rfe_k=, 3)    0.162205
(multiple_rfe_k=, 2)    0.162445
baseline                0.162624

(multiple_rfe_k=, 5)    0.157396
(multiple_rfe_k=, 4)    0.157447
(multiple_rfe_k=, 6)    0.157465
(multiple_rfe_k=, 7)    0.157465
(multiple_rfe_k=, 2)    0.157899
(multiple_rfe_k=, 3)    0.157910
basline                 0.158068

(multiple_rfe_k=, 4)    0.177443
(multiple_rfe_k=, 5)    0.177449
(multiple_rfe_k=, 6)    0.177509
(multiple_rfe_k=, 7)    0.177509
(multiple_rfe_k=, 3)    0.178001
(multiple_rfe_k=, 2)    0.178179
baseline                0.178470

Polynomial interaction only k=2:
(0.1614473947647387, 0.15759616934697873, 0.17765691560020958)

In [ ]:
all_cols = ['bathrooms', 'bedrooms', 'total_sqft', 'living_sqft',
       'full_bath', 'latitude', 'longitude', 'lot_sqft', 'roomcnt',
       'structure_value', 'assessed_value', 'land_value', 'taxamount','age', 'taxrate', 'transaction_month',
       'county_Los Angeles', 'county_Orange', 'county_Ventura', 'zip_code', 'zip_bin_insgfnt high', 'zip_bin_insgfnt low',
       'zip_bin_sgfnt high', 'zip_bin_sgfnt low', 'location_clusters',
       'age_clusters', 'area_clusters', 'size_clusters', 'value_clusters',
       'location_clusters_0', 'location_clusters_1', 'location_clusters_2',
       'location_clusters_3', 'age_clusters_0', 'age_clusters_1',
       'age_clusters_2', 'area_clusters_0', 'area_clusters_1',
       'area_clusters_2', 'size_clusters_0', 'size_clusters_1',
       'size_clusters_2', 'value_clusters_0', 'value_clusters_1',
       'value_clusters_2']

In [14]:
# set x and y
cols = ['living_sqft', 'structure_value', 'assessed_value', 'land_value',
       'county_Los Angeles', 'county_Orange', 'county_Ventura',
       'zip_bin_insgfnt high', 'zip_bin_sgfnt high', 'location_clusters_0',
       'location_clusters_2', 'location_clusters_3', 'age_clusters_0',
       'area_clusters_1', 'area_clusters_2', 'size_clusters_0',
       'value_clusters_0']
X_train = train[cols]
y_train = train.logerror

X_validate = validate[cols]
y_validate = validate.logerror

X_test = test[cols]
y_test = test.logerror

## Multiple  Regression + RFE

In [76]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()

# 1. Transform our X
rfe = RFE(lm, n_features_to_select=20)
rfe.fit(X_train, y_train)
print('selected top 6 features:', X_train.columns[rfe.support_])
X_train_rfe = rfe.transform(X_train)

selected top 6 features: Index(['living_sqft', 'structure_value', 'assessed_value', 'land_value',
       'county_Los Angeles', 'county_Orange', 'county_Ventura',
       'zip_bin_insgfnt high', 'zip_bin_sgfnt high', 'location_clusters',
       'area_clusters', 'value_clusters', 'location_clusters_0',
       'location_clusters_2', 'location_clusters_3', 'age_clusters_0',
       'area_clusters_1', 'area_clusters_2', 'size_clusters_0',
       'value_clusters_0'],
      dtype='object')


In [15]:
train_predictions = pd.DataFrame({
    'actual': train.logerror
}) 
validate_predictions = pd.DataFrame({
    'actual': validate.logerror
})
test_predictions = pd.DataFrame({
    'actual': test.logerror
})

In [16]:
train_predictions['baseline'] = y_train.mean()
validate_predictions['basline']=y_validate.mean()
test_predictions['baseline']=y_test.mean()

In [17]:
for k in range(2,16):
    lm = LinearRegression()
    # 1. Transform our X
    rfe = RFE(lm, n_features_to_select=k)
    rfe.fit(X_train, y_train)
    # 2. Use the transformed x in our model
    X_train_rfe = rfe.transform(X_train)
    X_validate_rfe = rfe.transform(X_validate)
    X_test_rfe = rfe.transform(X_test)
    lm.fit(X_train_rfe, y_train)
    
    # Make predictions
    train_predictions['multiple_rfe_k=', k] = lm.predict(X_train_rfe)
    validate_predictions['multiple_rfe_k=', k] = lm.predict(X_validate_rfe)
    test_predictions['multiple_rfe_k=', k] = lm.predict(X_test_rfe)
    k+=1

In [18]:
def calculate_rmse(y_predicted):
    return mean_squared_error(train_predictions.actual, y_predicted, squared = False)

train_predictions.apply(calculate_rmse).sort_values()

actual                   0.000000
(multiple_rfe_k=, 15)    0.161580
(multiple_rfe_k=, 14)    0.161580
(multiple_rfe_k=, 12)    0.161582
(multiple_rfe_k=, 13)    0.161582
(multiple_rfe_k=, 11)    0.161589
(multiple_rfe_k=, 10)    0.161605
(multiple_rfe_k=, 9)     0.161635
(multiple_rfe_k=, 7)     0.161640
(multiple_rfe_k=, 8)     0.161640
(multiple_rfe_k=, 6)     0.161650
(multiple_rfe_k=, 4)     0.161820
(multiple_rfe_k=, 5)     0.161820
(multiple_rfe_k=, 3)     0.161996
(multiple_rfe_k=, 2)     0.162445
baseline                 0.162624
dtype: float64

In [19]:
def calculate_rmse(y_predicted):
    return mean_squared_error(validate_predictions.actual, y_predicted, squared = False)

validate_predictions.apply(calculate_rmse).sort_values()

actual                   0.000000
(multiple_rfe_k=, 7)     0.157336
(multiple_rfe_k=, 8)     0.157336
(multiple_rfe_k=, 9)     0.157342
(multiple_rfe_k=, 11)    0.157371
(multiple_rfe_k=, 12)    0.157374
(multiple_rfe_k=, 13)    0.157374
(multiple_rfe_k=, 15)    0.157379
(multiple_rfe_k=, 14)    0.157382
(multiple_rfe_k=, 6)     0.157388
(multiple_rfe_k=, 10)    0.157402
(multiple_rfe_k=, 3)     0.157633
(multiple_rfe_k=, 4)     0.157658
(multiple_rfe_k=, 5)     0.157658
(multiple_rfe_k=, 2)     0.157899
basline                  0.158068
dtype: float64

In [20]:
def calculate_rmse(y_predicted):
    return mean_squared_error(test_predictions.actual, y_predicted, squared = False)

test_predictions.apply(calculate_rmse).sort_values()

actual                   0.000000
(multiple_rfe_k=, 6)     0.177436
(multiple_rfe_k=, 7)     0.177443
(multiple_rfe_k=, 8)     0.177443
(multiple_rfe_k=, 9)     0.177451
(multiple_rfe_k=, 11)    0.177485
(multiple_rfe_k=, 14)    0.177486
(multiple_rfe_k=, 12)    0.177486
(multiple_rfe_k=, 13)    0.177486
(multiple_rfe_k=, 15)    0.177490
(multiple_rfe_k=, 10)    0.177499
(multiple_rfe_k=, 4)     0.177781
(multiple_rfe_k=, 5)     0.177781
(multiple_rfe_k=, 3)     0.177918
(multiple_rfe_k=, 2)     0.178179
baseline                 0.178470
dtype: float64

## Polynomial Features

In [21]:
train_pred = pd.DataFrame({
    'actual': train.logerror
}) 
validate_pred = pd.DataFrame({
    'actual': validate.logerror
})
test_pred = pd.DataFrame({
    'actual': test.logerror
})
train_pred['baseline'] = y_train.mean()
validate_pred['basline']=y_validate.mean()
test_pred['basline']=y_test.mean()

In [22]:
from sklearn.preprocessing import PolynomialFeatures

# 1. Generate Polynomial Features, k=2
poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=False)
poly.fit(X_train)
X_train_poly = pd.DataFrame(
    poly.transform(X_train),
    columns=poly.get_feature_names(X_train.columns),
    index=X_train.index,
)

In [23]:
lm = LinearRegression()
lm.fit(X_train_poly, y_train)
X_validate_poly = poly.transform(X_validate)
X_test_poly = poly.transform(X_test)
train_pred['polynomial degree 2'] = lm.predict(X_train_poly)
validate_pred['polynomial degree 2'] = lm.predict(X_validate_poly)
test_pred['polynomial degree 2'] = lm.predict(X_test_poly)

In [24]:
train_rmse = mean_squared_error(train_pred.actual,train_pred['polynomial degree 2'], squared = False)
validate_rmse = mean_squared_error(validate_pred.actual,validate_pred['polynomial degree 2'], squared = False)
test_rmse = mean_squared_error(test_pred.actual,test_pred['polynomial degree 2'], squared = False)
train_rmse, validate_rmse, test_rmse

(0.16118663037140582, 0.15752054355182657, 0.17757655954543025)

In [25]:
# Interaction terms only

poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)
poly.fit(X_train)
X_train_poly = pd.DataFrame(
    poly.transform(X_train),
    columns=poly.get_feature_names(X_train.columns),
    index=X_train.index,
)
lm = LinearRegression()
lm.fit(X_train_poly, y_train)

X_validate_poly = poly.transform(X_validate)
X_test_poly = poly.transform(X_test)

train_pred['polynomial degree 2 only interaction'] = lm.predict(X_train_poly)
validate_pred['polynomial degree 2 only interaction'] = lm.predict(X_validate_poly)
test_pred['polynomial degree 2 only interaction'] = lm.predict(X_test_poly)

In [26]:
train_rmse = mean_squared_error(train_pred.actual,train_pred['polynomial degree 2 only interaction'], squared = False)
validate_rmse = mean_squared_error(validate_pred.actual,validate_pred['polynomial degree 2 only interaction'], squared = False)
test_rmse = mean_squared_error(test_pred.actual,test_pred['polynomial degree 2 only interaction'], squared = False)
train_rmse, validate_rmse, test_rmse

(0.1611927742715786, 0.15757468327292784, 0.17762433795538995)

In [ ]:
# k=3
poly = PolynomialFeatures(degree=3, include_bias=False, interaction_only=False)
poly.fit(X_train)
X_train_poly = pd.DataFrame(
    poly.transform(X_train),
    columns=poly.get_feature_names(X_train.columns),
    index=X_train.index,
)
lm = LinearRegression()
lm.fit(X_train_poly, y_train)
X_validate_poly = poly.transform(X_validate)
X_test_poly = poly.transform(X_test)
train_pred['polynomial degree 3'] = lm.predict(X_train_poly)
validate_pred['polynomial degree 3'] = lm.predict(X_validate_poly)
test_pred['polynomial degree 3'] = lm.predict(X_test_poly)

In [ ]:
train_rmse = mean_squared_error(train_pred.actual,train_pred['polynomial degree 3'], squared = False)
validate_rmse = mean_squared_error(validate_pred.actual,validate_pred['polynomial degree 3'], squared = False)
test_rmse = mean_squared_error(test_pred.actual,test_pred['polynomial degree 3'], squared = False)
train_rmse, validate_rmse, test_rmse

## Lasso-Lars

In [27]:
# create the model object
lars = LassoLars(alpha=1)

# fit the model to our training data
lars.fit(X_train, y_train)

# predict validate
X_train_pred_lars = lars.predict(X_train)
X_validate_pred_lars = lars.predict(X_validate)
X_test_pred_lars = lars.predict(X_test)
# Add lassolars predictions to our predictions DataFrame
train_pred['lasso_lars'] = X_train_pred_lars
validate_pred['lasso_lars'] = X_validate_pred_lars
test_pred['lasso_lars'] = X_test_pred_lars

In [28]:
train_rmse = mean_squared_error(train_pred.actual,train_pred['lasso_lars'], squared = False)
validate_rmse = mean_squared_error(validate_pred.actual,validate_pred['lasso_lars'], squared = False)
test_rmse = mean_squared_error(test_pred.actual,test_pred['lasso_lars'], squared = False)
train_rmse, validate_rmse, test_rmse

(0.16262411066536314, 0.1580787480680725, 0.1784697030682737)

## Generalized Linear Model

In [37]:
from sklearn.linear_model import TweedieRegressor

# create the model object
glm = TweedieRegressor(power=1, alpha=0)

# fit the model to our training data
glm.fit(X_train, y_train)

# predict train
X_train_predict_glm = glm.predict(X_train)
X_validate_predict_glm = glm.predict(X_validate)
X_test_predict_glm = glm.predict(X_test)
# Add lassolars predictions to our predictions DataFrame
train_pred['glm'] = X_train_predict_glm
validate_pred['glm'] = X_validate_predict_glm
test_pred['glm'] = X_test_predict_glm

ValueError: Some value(s) of y are out of the valid range for family TweedieDistribution